In [4]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

class JobinjaScraper:
    def __init__(self, base_url, headers):
        self.base_url = base_url
        self.headers = headers
        self.subpage_links = set()
        self.all_jobs_data = []

    def get_links(self):
        try:
            response = requests.get(self.base_url, headers=self.headers, verify=False, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            links = soup.find_all('a', href=True)

            for link in links:
                href = link['href']
                if href.startswith('/'):  # It's a relative link
                    full_url = self.base_url + href
                elif href.startswith('http'):
                    full_url = href
                else:
                    continue  # Skip if the href doesn't look like a valid link

                self.subpage_links.add(full_url)

        except requests.exceptions.SSLError as e:
            print(f"SSL error occurred: {e}")
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")

    def extract_subpage_text(self):
        for subpage_url in self.subpage_links:
            try:
                subpage_response = requests.get(subpage_url, headers=self.headers, verify=False, timeout=10)
                subpage_response.raise_for_status()

                subpage_soup = BeautifulSoup(subpage_response.text, 'html.parser')
                subpage_text = subpage_soup.get_text(strip=True)
                
                # Extract additional job data for sorting
                job_title = subpage_soup.find('h1', {'class': 'job-title'})
                job_type = subpage_soup.find('div', {'class': 'job-type'})
                job_location = subpage_soup.find('div', {'class': 'job-location'})
                company_name = subpage_soup.find('div', {'class': 'company-name'})
                contract_type = subpage_soup.find('div', {'class': 'contract-type'})
                work_experience = subpage_soup.find('div', {'class': 'work-experience'})
                min_salary = subpage_soup.find('div', {'class': 'min-salary'})

                job_data = {
                    "url": subpage_url,
                    "content_snippet": subpage_text[:1000],
                    "job_title": job_title.get_text(strip=True) if job_title else "N/A",
                    "job_type": job_type.get_text(strip=True) if job_type else "N/A",
                    "job_location": job_location.get_text(strip=True) if job_location else "N/A",
                    "company_name": company_name.get_text(strip=True) if company_name else "N/A",
                    "contract_type": contract_type.get_text(strip=True) if contract_type else "N/A",
                    "work_experience": work_experience.get_text(strip=True) if work_experience else "N/A",
                    "min_salary": min_salary.get_text(strip=True) if min_salary else "N/A",
                }
                self.all_jobs_data.append(job_data)

                print(f"URL: {subpage_url}\nContent Snippet: {subpage_text[:1000]}\n")
                time.sleep(1)  # To avoid overwhelming the server

            except requests.exceptions.RequestException as e:
                print(f"An error occurred while accessing {subpage_url}: {e}")

    def scrape(self):
        self.get_links()
        if not self.subpage_links:
            print("No links found to scrape.")
            return
        self.extract_subpage_text()

    def descriptive_statistics(self):
        # Generate descriptive statistics for the dataset
        df = pd.DataFrame(self.all_jobs_data)
        if not df.empty:
            print(df.describe(include='all'))
        else:
            print("No data available to generate statistics.")

    def sort_data(self):
        # Convert all_jobs_data to a DataFrame and sort by multiple columns
        df = pd.DataFrame(self.all_jobs_data)
        if not df.empty:
            df_sorted = df.sort_values(by=['job_title', 'job_type', 'job_location', 'contract_type', 'work_experience', 'min_salary'])
            print("Sorted Data:")
            print(df_sorted)
        else:
            print("No data available to sort.")

# Example usage
if __name__ == "__main__":
    url = "https://jobinja.ir/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    scraper = JobinjaScraper(url, headers)
    scraper.scrape()
    scraper.descriptive_statistics()
    scraper.sort_data()


C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


URL: https://jobinja.ir/jobs?filters%5Blocations%5D%5B%5D=%D8%A2%D8%B0%D8%B1%D8%A8%D8%A7%DB%8C%D8%AC%D8%A7%D9%86+%D8%BA%D8%B1%D8%A8%DB%8C
Content Snippet: آگهی استخدام آذربایجان غربی | استخدام آذربایجان غربی | کاریابی آذربایجان غربی | جابینجا
 | جابینجاامکان جدید:همین حالا رزومه خود را در کمتر از ۱۰ دقیقه بسازید،برو به رزومه‌سازJobinja | جابینجا | استخدام، آگهی استخدام، کاریابی، استخدامی جدید، بازار کار، نیازمندیها، ساخت رزومه رایگان، کار و استخدام دولتی، اشتغال، جستجوگر حرفه ای استخدام و شغلخانهجستجوی مشاغلرزومه‌ساز۵۰ شرکت برتربخش کارفرمایانورود کارجوثبت‌نام کارجوورود کارجوثبت‌نام کارجو۱جابینجا چطور به استخدام شدن من کمک می‌کند؟مشاهدهمشاهده بیشترجستجودرخواست‌های مننشان‌شده‌هاآگهی‌های پیشنهادیاز آگهی‌های جدید باخبر شوبهترین فرصت های شغلی در جابینجاهمه‌ی استان‌هاآذربایجان غربیتهرانخراسان رضویاصفهانالبرزفارسقمآذربایجان شرقیمازندرانگیلانخوزستانکرمانیزدقزوینمرکزیهرمزگانگلستانزنجانسیستان و بلوچستانبوشهرسمنانکرمانشاههمداناردبیللرستانکردستانخراسان شمالیایلامخراسان جنوبیچهارمحال بختیاریکهکیلوی

C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


URL: https://jobinja.ir/jobs?filters%5Bjob_categories%5D%5B%5D=%D9%85%D9%87%D9%86%D8%AF%D8%B3%DB%8C+%D9%BE%D9%84%DB%8C%D9%85%D8%B1
Content Snippet: استخدام کارشناس پلیمر | استخدام مهندس پلیمر | جابینجا
 | جابینجاامکان جدید:همین حالا رزومه خود را در کمتر از ۱۰ دقیقه بسازید،برو به رزومه‌سازJobinja | جابینجا | استخدام، آگهی استخدام، کاریابی، استخدامی جدید، بازار کار، نیازمندیها، ساخت رزومه رایگان، کار و استخدام دولتی، اشتغال، جستجوگر حرفه ای استخدام و شغلخانهجستجوی مشاغلرزومه‌ساز۵۰ شرکت برتربخش کارفرمایانورود کارجوثبت‌نام کارجوورود کارجوثبت‌نام کارجو۱جابینجا چطور به استخدام شدن من کمک می‌کند؟مشاهدهمشاهده بیشترجستجودرخواست‌های مننشان‌شده‌هاآگهی‌های پیشنهادیاز آگهی‌های جدید باخبر شوبهترین فرصت های شغلی در جابینجاهمه‌ی استان‌هاتهرانخراسان رضویاصفهانالبرزفارسقمآذربایجان شرقیمازندرانگیلانخوزستانکرمانیزدقزوینمرکزیهرمزگانگلستانزنجانسیستان و بلوچستانبوشهرآذربایجان غربیسمنانکرمانشاههمداناردبیللرستانکردستانخراسان شمالیایلامخراسان جنوبیچهارمحال بختیاریکهکیلویه و بویراحمدهمه‌ی دسته‌بندی‌هامهندسی پلیم

C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

URL: https://jobinja.ir/jobs?filters%5Bjob_categories%5D%5B%5D=%D8%AD%D9%85%D9%84+%D9%88+%D9%86%D9%82%D9%84
Content Snippet: استخدام کارشناس حمل و نقل | استخدام کارشناس لجستیک | جابینجا
 | جابینجاامکان جدید:همین حالا رزومه خود را در کمتر از ۱۰ دقیقه بسازید،برو به رزومه‌سازJobinja | جابینجا | استخدام، آگهی استخدام، کاریابی، استخدامی جدید، بازار کار، نیازمندیها، ساخت رزومه رایگان، کار و استخدام دولتی، اشتغال، جستجوگر حرفه ای استخدام و شغلخانهجستجوی مشاغلرزومه‌ساز۵۰ شرکت برتربخش کارفرمایانورود کارجوثبت‌نام کارجوورود کارجوثبت‌نام کارجو۱جابینجا چطور به استخدام شدن من کمک می‌کند؟مشاهدهمشاهده بیشترجستجودرخواست‌های مننشان‌شده‌هاآگهی‌های پیشنهادیاز آگهی‌های جدید باخبر شوبهترین فرصت های شغلی در جابینجاهمه‌ی استان‌هاتهرانخراسان رضویاصفهانالبرزفارسقمآذربایجان شرقیمازندرانگیلانخوزستانکرمانیزدقزوینمرکزیهرمزگانگلستانزنجانسیستان و بلوچستانبوشهرآذربایجان غربیسمنانکرمانشاههمداناردبیللرستانکردستانخراسان شمالیایلامخراسان جنوبیچهارمحال بختیاریکهکیلویه و بویراحمدهمه‌ی دسته‌بندی‌هاحمل و نقلفروش و بازاریابیوب

C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


URL: https://jobinja.ir/knowledge/employer-faq/pages/employer-account-faq
Content Snippet: سوالات متداول حساب کاربری کارفرمایان | جابینجاامکان جدید:همین حالا رزومه خود را در کمتر از ۱۰ دقیقه بسازید،برو به رزومه‌سازJobinja | جابینجا | استخدام، آگهی استخدام، کاریابی، استخدامی جدید، بازار کار، نیازمندیها، ساخت رزومه رایگان، کار و استخدام دولتی، اشتغال، جستجوگر حرفه ای استخدام و شغلخانهجستجوی مشاغلرزومه‌ساز۵۰ شرکت برتربخش کارفرمایانورود کارجوثبت‌نام کارجوورود کارجوثبت‌نام کارجو۱جابینجا چطور به استخدام شدن من کمک می‌کند؟مشاهدهمشاهده بیشترسوالات متداول کارفرمایانسوالات متداول حساب کاربری کارفرمایانسوالات متداول حساب کاربری کارفرمایانثبت‌نام و ورود به پنل شرکت:۱- اقدامات لازم جهت ایجاد پنل کارفرمایی چیست؟چنانچه در حال جذب نیرو هستید، وارد سایت جابینجا (www.jobinja.ir)
                شده و سپس از بخش کارفرمایان گزینه ثبت‌نام کارفرما را انتخاب نمایید و پس از تکمیل اطلاعات، از پنل ایجاد شده برای درج آگهی استخدام استفاده نمایید.۲- چنانچه پنل کارفرمایی شرکت موقتا غیرفعال شده چگونه آن را فعال کنم؟

C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


URL: https://jobinja.ir/jobs?filters%5Bkeywords%5D%5B0%5D=%DA%A9%D9%85%DA%A9+%D8%AD%D8%B3%D8%A7%D8%A8%D8%AF%D8%A7%D8%B1
Content Snippet: فرصت های شغلی کمک حسابدار
 | جابینجاامکان جدید:همین حالا رزومه خود را در کمتر از ۱۰ دقیقه بسازید،برو به رزومه‌سازJobinja | جابینجا | استخدام، آگهی استخدام، کاریابی، استخدامی جدید، بازار کار، نیازمندیها، ساخت رزومه رایگان، کار و استخدام دولتی، اشتغال، جستجوگر حرفه ای استخدام و شغلخانهجستجوی مشاغلرزومه‌ساز۵۰ شرکت برتربخش کارفرمایانورود کارجوثبت‌نام کارجوورود کارجوثبت‌نام کارجو۱جابینجا چطور به استخدام شدن من کمک می‌کند؟مشاهدهمشاهده بیشترجستجودرخواست‌های مننشان‌شده‌هاآگهی‌های پیشنهادیاز آگهی‌های جدید باخبر شوبهترین فرصت های شغلی در جابینجاهمه‌ی استان‌هاتهرانخراسان رضویاصفهانالبرزفارسقمآذربایجان شرقیمازندرانگیلانخوزستانکرمانیزدقزوینمرکزیهرمزگانگلستانزنجانسیستان و بلوچستانبوشهرآذربایجان غربیسمنانکرمانشاههمداناردبیللرستانکردستانخراسان شمالیایلامخراسان جنوبیچهارمحال بختیاریکهکیلویه و بویراحمدهمه‌ی دسته‌بندی‌هافروش و بازاریابیوب،‌ برنامه‌نویسی و نرم‌افزارمالی 